In [44]:
import os
import numpy as np
import cv2
from imutils import paths
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
from covid_detector.evaluate_covid19 import Evaluator

In [53]:
"""
Evaluation class that performs forward pass through trained models
"""
#%%
import os

import cv2
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import (
    AveragePooling2D, Dense, Dropout, Flatten, Input, BatchNormalization, ReLU
)
from tensorflow.keras.models import Model

from covid_detector.utils import CLASS_MAPPINGS


#%%
class Evaluator(object):

    def __init__(self, split=None, modality='pocus'):
        print("init")
        """
        Constructor of COVID model evaluator class.
        
        Arguments:
            modality {str} -- The data modality to be used. Chose from
                {'pocus', 'xray', 'ct'}, defaults to 'pocus'.
        """

        if modality not in ['pocus', 'xray', 'ct']:
            raise ValueError(f'Unknown modality provided: {modality}')

        self.modality = modality
        self.num_classes = 3 if modality == 'pocus' else 2
        # load correct weights
        if split is None:
            self.weights_path = os.path.join(
                '..', 'trained_models', modality + '.model'
            )
        else:
            # This is not the best, but the last one
            # self.weights_path = os.path.join(
            #     '..', 'trained_models', 'fold_' + split, 'pocus_fold_' + split + '.model'
            # )
            #
            # This is the best model:
            self.weights_path = os.path.join(
                '..', 'trained_models', 'fold_' + split, "variables", "variables"
            )
            print("Loading weights from ", self.weights_path)
            
        self.class_mappings = CLASS_MAPPINGS[self.modality]

        # load the VGG16 network, ensuring the head FC layer sets are left off
        baseModel = VGG16(
            weights="imagenet",
            include_top=False,
            input_tensor=Input(shape=(224, 224, 3))
        )

        # construct the head of the model that will be placed on top of the
        # the base model
        headModel = baseModel.output
        headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
        headModel = Flatten(name="flatten")(headModel)
        headModel = Dense(64)(headModel)
        headModel = BatchNormalization()(headModel)
        headModel = ReLU()(headModel)
        headModel = Dropout(0.5)(headModel)
        headModel = Dense(self.num_classes, activation="softmax")(headModel)

        # place the head FC model on top of the base model
        self.model = Model(inputs=baseModel.input, outputs=headModel)

        # restore weights
        try:
            self.model.load_weights(self.weights_path)
        except Exception:
            raise Exception('Error in model restoring.')

        print(f'Model restored. Class mappings are {self.class_mappings}')

    def __call__(self, image):
        """Performs a forward pass through the restored model

        Arguments:
            image {np.array} -- Input image on which prediction is performed. 
                No size requirements, but the image will be reshaped to 224 x
                224 pixels (aspec ratio is *not* preserved, so quadratic images
                are preferred).

        Returns:
            logits {np.array} -- Shape (self.num_classes,). Class probabilities
                for 2 (or 3) classes.
        """

        image = self.preprocess(image)
        return np.squeeze(self.model.predict(image))

    def preprocess(self, image):
        """Apply image preprocessing pipeline

        Arguments:
            image {np.array} -- Arbitrary shape, quadratic preferred

        Returns:
            np.array -- Shape 224,224. Normalized to [0, 1].
        """

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        image = np.expand_dims(np.array(image), 0) / 255.0
        return image


### Functions for prediction of video label

In [54]:
def majority_vote(preds, gt, vid_filenames):
    """
    Arguments:
    	preds: predicted classes (1-d list of class_names or integers)
        gt: list of same size with ground truth labels
        vid_filenames: list of filenames
    """
    preds = np.asarray(preds)
    gt = np.asarray(gt)
    vids = np.asarray([vid.split(".")[0] for vid in vid_filenames])
    vid_preds_out = []
    for v in np.unique(vids):
        preds_video = preds[vids==v]
        gt_check = np.unique(gt[vids==v])
        assert len(gt_check)==1, "gt must have the same label for the whole video"
        labs, pred_counts = np.unique(preds_video, return_counts=True)
        # take label that is predicted most often
        vid_pred = labs[np.argmax(pred_counts)]
        print("preds for video:", preds_video)
        # print(v, vid_pred, gt_check[0])
        vid_preds_out.append([v, vid_pred, gt_check[0]])
    print("video accuracy (majority):", accuracy_score([p[1] for p in vid_preds_out], [p[2] for p in vid_preds_out]))
    return vid_preds_out
        
def average_certainty(preds_logits, gt, vid_filenames):
    """
    Arguments:
    	preds: predicted classes (1-d list of class_names or integers)
        gt: list of same size with ground truth labels
        vid_filenames: list of filenames
    """
    preds_logits = np.asarray(preds_logits)
    gt = np.asarray(gt)
    vid_preds_out = []
    vids = np.array([vid.split(".")[0] for vid in vid_filenames])
    for v in np.unique(vids):
        preds_video_logits = preds_logits[vids==v]
        preds_video = np.sum(preds_video_logits, axis=0)
        print("preds for video:", preds_video)
        gt_check = np.unique(gt[vids==v])
        assert len(gt_check)==1, "gt must have the same label for the whole video"
        # take label that is predicted most often
        vid_pred = np.argmax(preds_video)
        print(v, vid_pred, gt_check[0])
        vid_preds_out.append([v, vid_pred, gt_check[0]])
    print("video accuracy (certainty):", accuracy_score([p[1] for p in vid_preds_out], [p[2] for p in vid_preds_out]))
    return vid_preds_out

## Evaluation script for cross validation

In [55]:
for i in range(1,2):
    print("------------- SPLIT ", i, "-------------------")
    # define data input path
    path = "../data_pocus/cross_validation_data/split"+str(i)
    
    train_labels, test_labels, test_files = [], [], []
    train_data, test_data = [], []

    # loop over the image paths (train and test)
    for imagePath in paths.list_images(path):

        # extract the class label from the filename
        label = imagePath.split(os.path.sep)[-2]

        # load the image, swap color channels, and resize it to be a fixed
        # 224x224 pixels while ignoring aspect ratio
        image = cv2.imread(imagePath)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # image = cv2.resize(image, (224, 224))

        # update the data and labels lists, respectively
        test_labels.append(label)
        test_data.append(image)
        test_files.append(imagePath.split(os.path.sep)[-1])

    # build ground truth data
    classes = ["covid", "pneunomia", "regular"]
    gt_class_idx = np.array([classes.index(lab) for lab in test_labels])
    
    model = Evaluator(split=str(i))
    
    logits = np.array([model(img) for img in test_data])
    predIdxs = np.argmax(logits, axis=1)
    
    print(
    classification_report(
        gt_class_idx, predIdxs, target_names=classes
        )
    )

    vid_preds_certainty = average_certainty(logits, gt_class_idx, np.array(test_files))
    vid_preds_majority = majority_vote(predIdxs, gt_class_idx, np.array(test_files))

init
Loading weights from  ../trained_models/fold_1/variables/variables
Model restored. Class mappings are ['covid', 'pneunomia', 'regular']
              precision    recall  f1-score   support

       covid       1.00      0.99      1.00       111
   pneunomia       0.96      1.00      0.98        22
     regular       1.00      1.00      1.00        49

    accuracy                           0.99       182
   macro avg       0.99      1.00      0.99       182
weighted avg       0.99      0.99      0.99       182

preds for video: [49.459686   0.8778944  4.66241  ]
Cov-Butterfly-COVID Lung 1 0 0
preds for video: [0.01668004 0.88386697 0.09945303]
Cov-C_ConvexProb_score3 1 0
preds for video: [48.888733   0.8593776  5.251895 ]
Cov-MSU-SkipLesions 0 0
preds for video: [0.52138484 9.024886   2.4537296 ]
Pneu-Atlas-pneumonia 1 1
preds for video: [4.1147294e-05 9.9864788e+00 1.3480580e-02]
Pneu-grep-pneumonia4 1 1
preds for video: [ 4.2803226  1.1730975 43.546577 ]
Reg-Butterfly 2 2
video 

# Tests

In [30]:
model = Evaluator(split=str(i))

init
../trained_models/fold_0/pocus_fold_0.model
Model restored. Class mappings are ['covid', 'pneunomia', 'regular']


In [31]:
logits = np.array([model(img) for img in test_data])
predIdxs = np.argmax(logits, axis=1)

print(
classification_report(
    gt_class_idx, predIdxs, target_names=classes
    )
)

vid_preds_certainty = average_certainty(logits, gt_class_idx, np.array(test_files))
vid_preds_majority = majority_vote(predIdxs, gt_class_idx, np.array(test_files))

              precision    recall  f1-score   support

       covid       0.56      0.56      0.56        63
   pneunomia       0.44      0.79      0.56        28
     regular       0.67      0.38      0.48        48

    accuracy                           0.54       139
   macro avg       0.56      0.57      0.53       139
weighted avg       0.57      0.54      0.53       139

Cov-Atlas+(44) 1 0
Cov-Atlas+(45) 1 0
Cov-Atlas-+(43) 1 0
Cov-Atlas-Day+1 2 0
Cov-Atlas-Day+2 0 0
Cov-Atlas-Day+3 0 0
Cov-Atlas-Day+4 0 0
Cov-B_ConvexProb_score1 0 0
Pneu-grep-pneumonia2 1 1
Reg-Atlas 0 2
Reg-Atlas-alines 2 2
Reg-Atlas-lungcurtain 1 2
Reg-Youtube 0 2
video accuracy (certainty): 0.46153846153846156
video accuracy (majority): 0.46153846153846156


In [33]:
vid_preds_certainty = average_certainty(logits, gt_class_idx, np.array(test_files))
vid_preds_majority = majority_vote(predIdxs, gt_class_idx, np.array(test_files))

preds for video: [0.7476117 7.1902814 1.0621074]
Cov-Atlas+(44) 1 0
preds for video: [0.8079691 2.2369947 1.9550362]
Cov-Atlas+(45) 1 0
preds for video: [0.6700281 4.4929104 2.8370612]
Cov-Atlas-+(43) 1 0
preds for video: [0.8921366  0.14128703 5.9665766 ]
Cov-Atlas-Day+1 2 0
preds for video: [11.983487   1.601364   3.4151487]
Cov-Atlas-Day+2 0 0
preds for video: [7.294277   0.18070902 0.5250141 ]
Cov-Atlas-Day+3 0 0
preds for video: [7.270071   0.07357544 0.65635365]
Cov-Atlas-Day+4 0 0
preds for video: [0.97913533 0.01976757 0.00109702]
Cov-B_ConvexProb_score1 0 0
preds for video: [ 2.7778866 23.352705   1.8694102]
Pneu-grep-pneumonia2 1 1
preds for video: [8.20206   2.8440855 0.9538548]
Reg-Atlas 0 2
preds for video: [ 0.16833213  1.628948   10.202721  ]
Reg-Atlas-alines 2 2
preds for video: [0.85913706 5.7818565  5.3590064 ]
Reg-Atlas-lungcurtain 1 2
preds for video: [5.5052576 4.1001887 2.3945541]
Reg-Youtube 0 2
video accuracy (certainty): 0.46153846153846156
preds for video: [1 

In [49]:
model2 = Evaluator(split=str(i))

init
../trained_models/fold_0/variables/variables
Model restored. Class mappings are ['covid', 'pneunomia', 'regular']


In [50]:
logits = np.array([model2(img) for img in test_data])
predIdxs = np.argmax(logits, axis=1)

print(
classification_report(
    gt_class_idx, predIdxs, target_names=classes
    )
)

vid_preds_certainty = average_certainty(logits, gt_class_idx, np.array(test_files))
vid_preds_majority = majority_vote(predIdxs, gt_class_idx, np.array(test_files))

              precision    recall  f1-score   support

       covid       0.70      0.56      0.62        63
   pneunomia       0.71      0.79      0.75        28
     regular       0.67      0.81      0.74        48

    accuracy                           0.69       139
   macro avg       0.69      0.72      0.70       139
weighted avg       0.69      0.69      0.69       139

preds for video: [0.41692764 5.862566   2.720506  ]
Cov-Atlas+(44) 1 0
preds for video: [0.2946806 1.1493942 3.5559251]
Cov-Atlas+(45) 2 0
preds for video: [0.2536777 2.318747  5.4275756]
Cov-Atlas-+(43) 2 0
preds for video: [1.0876949  0.30086467 5.61144   ]
Cov-Atlas-Day+1 2 0
preds for video: [11.281307   2.2195392  3.4991531]
Cov-Atlas-Day+2 0 0
preds for video: [6.9004602  0.37701795 0.72252214]
Cov-Atlas-Day+3 0 0
preds for video: [7.207155   0.11220858 0.68063533]
Cov-Atlas-Day+4 0 0
preds for video: [0.9263738  0.06457515 0.0090511 ]
Cov-B_ConvexProb_score1 0 0
preds for video: [ 3.1745064 23.195557   1.